# <CENTER>**`VII.` SOCIETY RECOMMENDATION ENGINE**<CENTER>
---

#### Creating Recommendation Engine to recommend Real-Estate Properties based on "Society".
#### **`Recommendation Engine will be combination of 3 Engines:`**
1. Recommend Society with similar **Property pricing**
2. Recommend Society with similar **Nearby Locations**
3. Recommend Society with similar **Facilities/Amenities**

---

In [137]:
## Importing all necessary tools:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")        ## Hides all warnings

In [138]:
DF = pd.read_csv("Gurgaon_Real_Estate_data_for_recommendation_engine.csv", index_col = "Unnamed: 0")

## Removing all NAN records from Society column:
df = DF[DF['Society'] != '-'].reset_index(drop=True)
df.head(2)

,Flat,Sector_Locality,Locality,Seller_Builder,Sector_Locality_Price_Density,EMI,EMI_in_rupees,Built_Up_Area,Built_up_area_in_sqft,Avg_Price,Avg_price_rupee_per_sqft,Age_of_property,Age_of_property_in_years,Age_Category,Floor,Floor_number,Building_height,Floor_Category,Building_Height_Category,Facing,Furnishing,Society,Brokerage,Brokerage_in_rupees,Price,Price_in_rupees,Bedrooms,Bathrooms,Parking,Covered_parking,Open_parking,Balcony,Advertised,Amenities,Nearby_landmarks,Prop_description,Link,Sector_Amenity_Score
0,2 BHK Flat,Sector 86,New Gurugram,Pyramid Infratech Private Limited,10620.0,EMI starts at ₹36.01 K,36010,593 sq.ft,593,₹11.47 K/sq.ft,11470.0,1 Years Old,1,Newer Property,12 of 14,12,14,Upper Floor,Shorter Building,East facing,Semi Furnished,Pyramid Elite,"68,000",68000,68.0 L,6800000,2,2,1 Open Parking,0,1,1,More than a month ago,"['Lift', 'Power Backup', 'Garden', 'Sports', '...","[['School', ""St. Xavier's High School""], ['Hos...",Looking for a 2 BHK Flat for sale in Gurgaon? ...,https://housing.com/in/buy/resale/page/1761033...,2.0
1,2 BHK Flat,Sector 86,New Gurugram,Pyramid Infratech Private Limited,10620.0,EMI starts at ₹33.36 K,33360,690 sq.ft,690,₹9.13 K/sq.ft,9130.0,1 Years Old,1,Newer Property,4 of 15,4,15,Lower Floor,Medium Building,-,Unfurnished,Pyramid Elite,"63,000",63000,63.0 L,6300000,2,2,No Parking,0,0,1,17 days ago,NaN,"[['School', ""St. Xavier's High School""], ['Hos...",Best 2 BHK Flat for modern-day lifestyle is no...,https://housing.com/in/buy/resale/page/1681859...,2.0


#### `NOTE:` There are societies with SIMILAR names but at different locations. Lets create a new feature where Societies have their localities attached:

In [141]:
vals = df['Society'] + ", " + df['Sector_Locality']
df.insert(0, "Society_Loc", vals, allow_duplicates = False)
df.head(2)

,Society_Loc,Flat,Sector_Locality,Locality,Seller_Builder,Sector_Locality_Price_Density,EMI,EMI_in_rupees,Built_Up_Area,Built_up_area_in_sqft,Avg_Price,Avg_price_rupee_per_sqft,Age_of_property,Age_of_property_in_years,Age_Category,Floor,Floor_number,Building_height,Floor_Category,Building_Height_Category,Facing,Furnishing,Society,Brokerage,Brokerage_in_rupees,Price,Price_in_rupees,Bedrooms,Bathrooms,Parking,Covered_parking,Open_parking,Balcony,Advertised,Amenities,Nearby_landmarks,Prop_description,Link,Sector_Amenity_Score
0,"Pyramid Elite, Sector 86",2 BHK Flat,Sector 86,New Gurugram,Pyramid Infratech Private Limited,10620.0,EMI starts at ₹36.01 K,36010,593 sq.ft,593,₹11.47 K/sq.ft,11470.0,1 Years Old,1,Newer Property,12 of 14,12,14,Upper Floor,Shorter Building,East facing,Semi Furnished,Pyramid Elite,"68,000",68000,68.0 L,6800000,2,2,1 Open Parking,0,1,1,More than a month ago,"['Lift', 'Power Backup', 'Garden', 'Sports', '...","[['School', ""St. Xavier's High School""], ['Hos...",Looking for a 2 BHK Flat for sale in Gurgaon? ...,https://housing.com/in/buy/resale/page/1761033...,2.0
1,"Pyramid Elite, Sector 86",2 BHK Flat,Sector 86,New Gurugram,Pyramid Infratech Private Limited,10620.0,EMI starts at ₹33.36 K,33360,690 sq.ft,690,₹9.13 K/sq.ft,9130.0,1 Years Old,1,Newer Property,4 of 15,4,15,Lower Floor,Medium Building,-,Unfurnished,Pyramid Elite,"63,000",63000,63.0 L,6300000,2,2,No Parking,0,0,1,17 days ago,NaN,"[['School', ""St. Xavier's High School""], ['Hos...",Best 2 BHK Flat for modern-day lifestyle is no...,https://housing.com/in/buy/resale/page/1681859...,2.0


#### **`Selecting only Top 300 Societies with most number of Listed Properties:`**

In [144]:
top_300_soc = df['Society_Loc'].value_counts().head(300).index
df = df[df['Society_Loc'].isin(top_300_soc)].reset_index(drop=True)

### **NOTE:**
- We have good data for Property pricing based Society recommendation.
- But have limited data for Nearby Locations and Facilities/Amenities based Society recommendation.
- Because only 4 types of Locations are available - Hospitals, Schools, Restaurants and Metro Stations. And Amenities data seems repetetive and least variable with lot of pre-processing needs.

<br><br>

---
# `I.` **Property Pricing Based Society Recommendation**

In [42]:
def pricing_info_scrapper(data):
    Soc_Record = pd.Series()          ## Series to store overall society pricing info...

    ## Looping through all flat types avilable in the Society:
    for flat in data['Flat'].unique():
        flat_name = "_".join(flat.split()[:2])

        ## Filtering out a particulat flat type data:
        temp_df = data[data['Flat'] == flat]
        ## Creating series with pricing info for that flat type:
        temp_ser = pd.Series([temp_df.shape[0] , 
                              temp_df['Built_up_area_in_sqft'].min(),
                              temp_df['Built_up_area_in_sqft'].max(),
                              temp_df['Avg_price_rupee_per_sqft'].min(),
                              temp_df['Avg_price_rupee_per_sqft'].max(),
                              temp_df['Price_in_rupees'].min(),
                              temp_df['Price_in_rupees'].max()], 
                             
                             index = [flat_name, flat_name+"_area_min", flat_name+"_area_max",
                                      flat_name+"_PD_min", flat_name+"_PD_max",
                                      flat_name+"_cost_min", flat_name+"_cost_max"])

        Soc_Record = pd.concat([Soc_Record, temp_ser])  # Appending the particular Flat info to Society Record...

    
    Soc_Record = pd.concat([Soc_Record, pd.Series([data['Sector_Amenity_Score'].median()], 
                                                  index = ['Sector_Amenity_Score'])])
    return pd.DataFrame(Soc_Record)


Pricing_data = df.groupby(by = 'Society_Loc').apply(pricing_info_scrapper).unstack(fill_value=0)
Pricing_data.columns = [i[1] for i in Pricing_data.columns.values]

Pricing_data.sample(5)

,1_BHK,1_BHK_PD_max,1_BHK_PD_min,1_BHK_area_max,1_BHK_area_min,1_BHK_cost_max,1_BHK_cost_min,1_RK,1_RK_PD_max,1_RK_PD_min,1_RK_area_max,1_RK_area_min,1_RK_cost_max,1_RK_cost_min,2.5_BHK,2.5_BHK_PD_max,2.5_BHK_PD_min,2.5_BHK_area_max,2.5_BHK_area_min,2.5_BHK_cost_max,2.5_BHK_cost_min,2_BHK,2_BHK_PD_max,2_BHK_PD_min,2_BHK_area_max,2_BHK_area_min,2_BHK_cost_max,2_BHK_cost_min,3.5_BHK,3.5_BHK_PD_max,3.5_BHK_PD_min,3.5_BHK_area_max,3.5_BHK_area_min,3.5_BHK_cost_max,3.5_BHK_cost_min,3_BHK,3_BHK_PD_max,3_BHK_PD_min,3_BHK_area_max,3_BHK_area_min,3_BHK_cost_max,3_BHK_cost_min,4.5_BHK,4.5_BHK_PD_max,4.5_BHK_PD_min,4.5_BHK_area_max,4.5_BHK_area_min,4.5_BHK_cost_max,4.5_BHK_cost_min,4_BHK,4_BHK_PD_max,4_BHK_PD_min,4_BHK_area_max,4_BHK_area_min,4_BHK_cost_max,4_BHK_cost_min,5_BHK,5_BHK_PD_max,5_BHK_PD_min,5_BHK_area_max,5_BHK_area_min,5_BHK_cost_max,5_BHK_cost_min,6_BHK,6_BHK_PD_max,6_BHK_PD_min,6_BHK_area_max,6_BHK_area_min,6_BHK_cost_max,6_BHK_cost_min,Sector_Amenity_Score
Society_Loc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"ROF Amaltas, Sector 92",6.0,11430.0,7100.0,500.0,350.0,4000000.0,3500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,11660.0,6200.0,1120.0,643.0,7500000.0,6200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00
"Tata Gurgaon Gateway, Sector 112",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,22080.0,15520.0,1579.0,1200.0,26500000.0,24500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,15510.0,13300.0,2870.0,2120.0,38500000.0,30000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.67
"DLF New Town Heights 1, Sector 86",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7370.0,7370.0,190.0,190.0,1400000.0,1400000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15270.0,15270.0,1277.0,1277.0,19500000.0,19500000.0,2.0,10620.0,10620.0,1930.0,1930.0,20500000.0,20500000.0,15.0,13870.0,10050.0,1930.0,1550.0,21500000.0,19400000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,10980.0,10190.0,2536.0,2356.0,27500000.0,24000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00
"Ambience Lagoon, DLF Phase 3",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,24850.0,24850.0,2656.0,2656.0,66000000.0,66000000.0,8.0,25480.0,15710.0,3250.0,1570.0,52500000.0,33000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.67
"M3M Merlin, Sector 67",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19620.0,19620.0,2727.0,2727.0,53500000.0,53500000.0,23.0,32100.0,14320.0,2726.0,1850.0,61000000.0,29000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,28410.0,17860.0,3297.0,2200.0,65000000.0,50000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.00


In [43]:
## Scaling Features:
scaler = StandardScaler()
scaled_pricing_data = pd.DataFrame(scaler.fit_transform(Pricing_data),    ## converting the scaled data into DF...
                                  index = Pricing_data.index, columns = Pricing_data.columns)    

scaled_pricing_data.sample(3)

,1_BHK,1_BHK_PD_max,1_BHK_PD_min,1_BHK_area_max,1_BHK_area_min,1_BHK_cost_max,1_BHK_cost_min,1_RK,1_RK_PD_max,1_RK_PD_min,1_RK_area_max,1_RK_area_min,1_RK_cost_max,1_RK_cost_min,2.5_BHK,2.5_BHK_PD_max,2.5_BHK_PD_min,2.5_BHK_area_max,2.5_BHK_area_min,2.5_BHK_cost_max,2.5_BHK_cost_min,2_BHK,2_BHK_PD_max,2_BHK_PD_min,2_BHK_area_max,2_BHK_area_min,2_BHK_cost_max,2_BHK_cost_min,3.5_BHK,3.5_BHK_PD_max,3.5_BHK_PD_min,3.5_BHK_area_max,3.5_BHK_area_min,3.5_BHK_cost_max,3.5_BHK_cost_min,3_BHK,3_BHK_PD_max,3_BHK_PD_min,3_BHK_area_max,3_BHK_area_min,3_BHK_cost_max,3_BHK_cost_min,4.5_BHK,4.5_BHK_PD_max,4.5_BHK_PD_min,4.5_BHK_area_max,4.5_BHK_area_min,4.5_BHK_cost_max,4.5_BHK_cost_min,4_BHK,4_BHK_PD_max,4_BHK_PD_min,4_BHK_area_max,4_BHK_area_min,4_BHK_cost_max,4_BHK_cost_min,5_BHK,5_BHK_PD_max,5_BHK_PD_min,5_BHK_area_max,5_BHK_area_min,5_BHK_cost_max,5_BHK_cost_min,6_BHK,6_BHK_PD_max,6_BHK_PD_min,6_BHK_area_max,6_BHK_area_min,6_BHK_cost_max,6_BHK_cost_min,Sector_Amenity_Score
Society_Loc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Godrej Meridien, Sector 106",-0.225647,-0.341515,-0.333845,-0.337479,-0.323658,-0.295646,-0.317665,-0.155451,-0.197531,-0.195658,-0.153603,-0.151452,-0.140389,-0.13982,-0.299767,-0.446367,-0.448546,-0.462664,-0.460339,-0.435031,-0.432223,0.559220,1.057948,1.143316,0.995683,1.156887,1.226951,1.184933,-0.054036,1.653919,1.876418,1.438837,1.530169,1.595947,1.797869,2.355138,0.358925,0.604324,1.202826,0.399241,1.028508,0.330417,-0.196452,-0.260600,-0.259162,-0.297321,-0.296872,-0.237632,-0.236936,1.166142,1.419704,0.934792,0.999279,0.639322,0.902456,0.521072,-0.179173,-0.211655,-0.210831,-0.220603,-0.220252,-0.206675,-0.210932,-0.057831,-0.057831,-0.057831,-0.057831,-0.057831,-0.057831,-0.057831,-0.852256
"Spaze Privvy The Address, Sector 93",-0.225647,-0.341515,-0.333845,-0.337479,-0.323658,-0.295646,-0.317665,-0.155451,-0.197531,-0.195658,-0.153603,-0.151452,-0.140389,-0.13982,-0.299767,-0.446367,-0.448546,-0.462664,-0.460339,-0.435031,-0.432223,-0.164801,0.584070,0.320256,0.921458,0.810545,0.491300,0.411705,0.276795,0.739263,0.868049,1.431597,1.522614,0.704012,0.821840,0.204720,-0.170034,-0.269585,0.486313,0.506779,-0.130308,-0.236967,0.710249,1.588041,1.643523,4.783200,4.802794,2.178751,2.239289,0.448761,0.300272,0.386533,2.012912,1.322096,0.785857,0.439898,-0.179173,-0.211655,-0.210831,-0.220603,-0.220252,-0.206675,-0.210932,-0.057831,-0.057831,-0.057831,-0.057831,-0.057831,-0.057831,-0.057831,-0.643919
"Vatika Lifestyle Homes, Sector 83",-0.225647,-0.341515,-0.333845,-0.337479,-0.323658,-0.295646,-0.317665,-0.155451,-0.197531,-0.195658,-0.153603,-0.151452,-0.140389,-0.13982,0.416808,1.516884,1.786241,2.297725,2.376662,1.794826,1.833904,-0.371664,0.373739,0.516739,0.972961,0.784569,0.442257,0.477049,-0.384867,-0.592509,-0.600172,-0.631834,-0.630562,-0.584338,-0.587979,0.978870,-0.125490,-0.137175,0.341845,0.076625,-0.306650,-0.147380,-0.196452,-0.260600,-0.259162,-0.297321,-0.296872,-0.237632,-0.236936,-0.507746,-0.751347,-0.741177,-0.807459,-0.813137,-0.687537,-0.676251,-0.179173,-0.211655,-0.210831,-0.220603,-0.220252,-0.206675,-0.210932,-0.057831,-0.057831,-0.057831,-0.057831,-0.057831,-0.057831,-0.057831,-0.230353


<br><br>
### **Creating "Society Similarity Matrix" using Cosine Similerity Score between Societies:**

In [45]:
## Computing Similarity Score between Socities based on Scaled Pricing Information:
Pricing_similarity_matrix = pd.DataFrame(cosine_similarity(scaled_pricing_data, scaled_pricing_data),
                                        index = scaled_pricing_data.index, columns = scaled_pricing_data.index)
Pricing_similarity_matrix.head(4)

Society_Loc,"AIPL The Peaceful Homes, Sector 70A","AIPL Zen Residences, Sector 70A","ATS Kocoon, Sector 109","ATS Marigold, Sector 89A","ATS Tourmaline, Sector 109","ATS Triumph, Sector 104","AVL 36, Sector 36","Adani Aangan, Sector 89A","Adani Brahma Samsara Vilasa, Sector 63","Adani M2K Oyster Grande, Sector 102","Adani Oyster Greens, Sector 102","Ambience Creacions, Sector 22A","Ambience Lagoon, DLF Phase 3","Anant Raj Maceo, Sector 91","Ansal Heights 86, Sector 86","Ansal Heights, Sector 92","Ansal Sushant Estate, Sector 52","Ansal Valley View Estate, Gwal Pahari","Ansals Highland Park, Sector 103","Apex Our Homes 1, Sector 37C","Apex Our Homes, Sector 37C","Ardee Palm Grove Heights, Sector 52","Ashiana Anmol Plaza Phase I, Sector 33","Ashiana Anmol, Dhunela","Ashiana Landcraft The Center Court, Sector 88A","Ashiana Mulberry, Sector 2","Assotech Blith, Sector 99","BPTP Amstoria Country Floor, Sector 102","BPTP Park Serene, Sector 37D","BPTP Terra, Sector 37D","Bestech Altura, Sector 79","Bestech Park View Altura, Sector 79","Bestech Park View Ananda, Sector 81","Bestech Park View Grand Spa, Sector 81","Bestech Park View Residency, Palam Vihar","Bestech Park View Sanskruti, Sector 92","Bestech Park View Spa Next, Sector 67","Bhagwanti Apartment, Sector 56","Birla Navya Gurugram, Kadarpur","Breez Global Heights, Sector 33","Brisk Lumbini Terrace Homes, Sector 109","CGHS City Heights, Sector 39","CGHS Millenium Residency, Sector 47","CGHS Progressive Apartments, Sector 55","CGHS Sanskriti Apartments, Sector 43","CGHS Tarika CGHS, Sushant Lok Phase 1","CGHS The New Greenwood, Sector 52","CGHS Viman Apartment, Sector 9A","CHD Avenue 71, Sector 72","Central Aqua Front Towers, Sector 33","Central Park Aqua Front Towers, Sector 33","Central Park Flamingo Floors, Sector 32","Central Park Flower Valley, Sector 33","Chintels Serenity Phase 1 Tower 3 4 5 Pocket B, Sector 109","Chintels Serenity Pocket B Phase II, Sector 109","Conscient Elevate, Sector 59","Conscient Habitat, Sector 99A","Conscient Heritage Max, Sector 102","Conscient Hines Elevate, Sector 59","Corona Optus, Sector 37C","Cosmos Express 99, Sector 99","DLF Carlton Estate, Sector 53","DLF Exclusive Floors, Sector 54","DLF Express Greens, Manesar","DLF Garden City Enclave Floors, Sector 93","DLF Garden City Independent Floors, Sector 92","DLF New Town Heights 1, Sector 86","DLF New Town Heights 1, Sector 90","DLF New Town Heights 2, Sector 86","DLF New Town Heights 3, Sector 91","DLF Oakwood Estate, Sector 25","DLF Park Place, DLF Phase 5","DLF Regal Gardens, Sector 90","DLF Regency Park 1, Sector 43","DLF Richmond Park, Sector 43","DLF Ridgewood Estate, DLF Phase 4","DLF Silver Oaks, DLF Phase 1","DLF The Crest, DLF Phase 5","DLF The Primus, Sector 82A","DLF The Skycourt, Sector 86","DLF The Ultima Phase II, Sector 81","DLF The Ultima, Sector 81","DLF Wellington Estate, Sector 53","DLF Westend Heights, Sector 53","Eldeco Acclaim, Sector 2","Eldeco Accolade, Sector 2","Emaar Digi Homes, Sector 62","Emaar Emerald Classic, Sector 65","Emaar Emerald Estate, Sector 65","Emaar Emerald Floors Premier, Sector 65","Emaar Emerald Floors, Sector 65","Emaar Emerald Hill, Sector 65","Emaar Gurgaon Greens, Sector 102","Emaar Imperial Gardens, Sector 102","Emaar MGF The Palm Drive, Sector 66","Emaar Palm Gardens, Sector 83","Emaar Palm Heights, Sector 77","Emaar Palm Hills, Sector 77","Emaar Palm Premier, Sector 77","Emaar Palm Select, Sector 77","Emaar Palm Terraces Select, Sector 66","Emaar The Enclave, Sector 66","Experion The Heartsong, Sector 108","Experion Windchants, Sector 112","GLS Arawali Homes, Sector 4","GLS Avenue 51, Sector 92","GLS South Avenue, Sector 92","GPL Eden Heights, Sector 70","Godrej 101, Sector 79","Godrej Air Phase I, Sector 85","Godrej Air, Sector 85","Godrej Aria, Sector 79","Godrej Frontier, Sector 80","Godrej Icon, Sector 88A","Godrej Meridien, Sector 106","Godrej Nature Plus, Sector 33","Godrej Oasis, Sector 88A","Godrej Summit, Sector 104","Gurgaon One 

<br><br>
### **Finally Recommending Societies based on Pricing Similarity Scores:**

In [50]:
## Using the above Pricing Similarity Matrix for Recommending Top 10 Similar Socities:
def recommend_societies_by_pricing(Society_name, Pricing_similarity_matrix):
    societies = Pricing_similarity_matrix[Society_name].sort_values(ascending = False).iloc[1:11].index
    return pd.DataFrame(societies, index = range(1,11))

recommend_societies_by_pricing("ATS Kocoon, Sector 109", Pricing_similarity_matrix)

,Society_Loc
1,"ATS Tourmaline, Sector 109"
2,"Solutrean Caladium, Sector 109"
3,"Bestech Altura, Sector 79"
4,"DLF New Town Heights 1, Sector 90"
5,"DLF Garden City Independent Floors, Sector 92"
6,"DLF Regal Gardens, Sector 90"
7,"Mapsko Mount Ville, Sector 79"
8,"Bestech Park View Grand Spa, Sector 81"
9,"DLF New Town Heights 3, Sector 91"
10,"Puri Diplomatic Greens, Sector 110A"


In [303]:
## Saving the Similarity Matrix as CSV:
Pricing_similarity_matrix.to_csv("REC_1_Pricing_similarity_matrix.csv")

<br><br><br>

---

# `II.` **Facilities Based Society Recommendation**

In [66]:
## Remving all the NAN records from Amenities Feature:
df_new = df[(~ df['Amenities'].isnull())].reset_index(drop=True)
df_new.head(3)

,Society_Loc,Flat,Sector_Locality,Locality,Seller_Builder,Sector_Locality_Price_Density,EMI,EMI_in_rupees,Built_Up_Area,Built_up_area_in_sqft,Avg_Price,Avg_price_rupee_per_sqft,Age_of_property,Age_of_property_in_years,Age_Category,Floor,Floor_number,Building_height,Floor_Category,Building_Height_Category,Facing,Furnishing,Society,Brokerage,Brokerage_in_rupees,Price,Price_in_rupees,Bedrooms,Bathrooms,Parking,Covered_parking,Open_parking,Balcony,Advertised,Amenities,Nearby_landmarks,Prop_description,Link,Sector_Amenity_Score
0,"Pyramid Elite, Sector 86",2 BHK Flat,Sector 86,New Gurugram,Pyramid Infratech Private Limited,10620.0,EMI starts at ₹36.01 K,36010,593 sq.ft,593,₹11.47 K/sq.ft,11470.0,1 Years Old,1,Newer Property,12 of 14,12,14,Upper Floor,Shorter Building,East facing,Semi Furnished,Pyramid Elite,"68,000",68000,68.0 L,6800000,2,2,1 Open Parking,0,1,1,More than a month ago,"['Lift', 'Power Backup', 'Garden', 'Sports', '...","[['School', ""St. Xavier's High School""], ['Hos...",Looking for a 2 BHK Flat for sale in Gurgaon? ...,https://housing.com/in/buy/resale/page/1761033...,2.0
1,"Experion The Heartsong, Sector 108",2 BHK Flat,Sector 108,Dwarka Expressway Belt,Experion Developers,18375.0,EMI starts at ₹74.47 K,74470,1000 sq.ft,1000,₹15 K/sq.ft,15000.0,2 Year Old,2,Newer Property,9 of 26,9,26,Middle Floor,Taller Building,North-East facing,Semi Furnished,Experion The Heartsong,1.5 Lacs,150000,1.5 Cr,15000000,2,2,1 Covered and 1 Open Parking,1,1,3,More than a month ago,"['Amphitheater', 'Cricket Pitch', 'Gazebo', 'S...","[['School', 'The Shikshiyan School'], ['Hospit...","2 BHK Flat for sale in Sector 108, Gurgaon - c...",https://housing.com/in/buy/resale/page/1767484...,0.0
2,"ROF Aalayas 1, Sector 102",3 BHK Flat,Sector 102,Dwarka Expressway Belt,ROF Infratech,13230.0,EMI starts at ₹39.19 K,39190,645 sq.ft,645,₹11.47 K/sq.ft,11470.0,1 Years Old,1,Newer Property,5 of 14,5,14,Middle Floor,Shorter Building,South facing,Semi Furnished,ROF Aalayas 1,"74,000",74000,74.0 L,7400000,3,2,1 Covered and 1 Open Parking,1,1,1,More than a month ago,"['Stove', 'Gas Pipeline', 'Cupboard', 'Pet all...","[['School', 'Delhi Public School'], ['Hospital...","3 BHK Flat for sale in Sector 102, Gurgaon - c...",https://housing.com/in/buy/resale/page/1734127...,2.0


In [177]:
def facility_info_scrapper(data):
    Soc_facilities = {}                              # Dictionary of amenities with count in the society...

    values = data['Amenities'].str.replace("\\xa0", " ").values   # Fixing irregular parts of string 

    for amenity_list in values:
        temp = amenity_list[1:-1].split(", ")

        for i in temp:
            j = i[1:-1]
            if j not in Soc_facilities:
                Soc_facilities[j] = 1
            else:
                Soc_facilities[j] += 1

    Soc_Record = pd.Series(Soc_facilities)            # Series to store overall society facilities info...
    
    return pd.DataFrame(Soc_Record)


Facility_data = df_new.groupby(by = 'Society_Loc').apply(facility_info_scrapper).unstack(fill_value=0)
Facility_data.columns = [i[1] for i in Facility_data.columns.values]

Facility_data.sample(2)

,10Fan,10Light,11Fan,11Light,12Fan,12Light,13Light,14Light,15Light,16Light,17Light,18Light,19Light,1AC,1Bed,1Fan,1Geyser,1Geyser Count,1Light,1TV,1Wardrobe,20Light,21Light,22Light,23Light,24X7 Water Supply,24x7 Security,25Light,26Light,2AC,2Bed,2Cupboard Count,2Fan,2Geyser,2Geyser Count,2Light,2Wardrobe,31Light,3AC,3Bed,3Cupboard Count,3Fan,3Geyser,3Light,3Wardrobe,40Light,4AC,4Fan,4Geyser,4Light,4Wardrobe,50Light,5AC,5Fan,5Geyser,5Light,5Wardrobe,6AC,6Fan,6Geyser,6Light,6Wardrobe,7AC,7Fan,7Geyser,7Light,8AC,8Fan,8Light,8Wardrobe,9Fan,9Light,AC,Amphitheater,Badminton Court,Banquet Hall,Bar/Chill-out Lounge,Barbecue Area,Bed,Billiards / Snooker Table,Bus Shelter,Business Suites,CCTV,Car Parking,Card Room,Central Cooling System,Changing Room,Children's Play Area,Chimney,Closed Car Parking,Club House,Club Rooftop,Community Buildings,Community Hall,Concierge Service,Creche/Day Care,Cricket Pitch,Cupboard,Cycling & Jogging Track,Dining Table,Electrification(Transforme,Entrance Lobby,Fire Alarm,Fire Fighting System,Fire Sprinklers,Fitness Center,Flower Garden,Food Court,Football Field,Fountains,Fridge,Garbage Disposal,Garden,Gas Pipeline,Gated Community,Gazebo,Geyser,Grocery Shop,Gym,Gymnasium,Has vastu compliant,Health Facilities,Hockey Ground,Indoor Games,Intercom,Internet / Wi-Fi,Jogging Track,Kid's Pool,Kids Area,Landscaping & Tree Planting,Letter Box,Lift,Lift(s),Maintenance Staff,Medical Store/Pharmacy,Microwave,Mini Theatre,Modular Kitchen,Multipurpose Room,Natural Pond,Parking,Partial Power Backup,Paved Compound,Pet Grooming,Pet allowed,Power Backup,RO Water System,Rain Water Harvesting,Reading Lounge,Recreation Facilities,Reflexology Park,Regular Water Supply,Sauna Bath,School,Security Cabin,Senior Citizen Siteout,Servant Room,Sewage Treatment Plant,Shopping Mall,Skating Rink,Sofa,Solar Lighting,Solid Waste Management And Disposal,Spa,Sports,Sports Area,Sports Facility,Squash Court,Stove,Street Lighting,Sun Bathing,Swimming Pool,TV,Table Tennis,Temple,Tennis Court,Theme Park,Vaastu Compliant,Valet Parking,Vastu Compliant,Video Door Security,Visitor Parking,Volleyball Court,Waiting Lounge,Washing Machine,Water Conservatio,Water Purifier,Water Softner Plant,Water Storage,Yoga / Meditation Area,ain water Harvesting,olar Energy etc)
Society_Loc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Shree Vardhman Flora, Sector 90",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,2,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
"Pivotal Paradise, Sector 62",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,2,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,2,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,1,2,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0


#### **Removing repetetive columns :**

In [178]:
lights = ['10Light', '11Light', '12Light', '13Light', '14Light', '15Light', '16Light', '17Light', '18Light', 
          '19Light', '1Geyser Count', '1Light', '20Light', '21Light', '22Light', '23Light', '25Light', '26Light',
          '2Cupboard Count', '2Geyser Count', '2Light', '31Light', '3Cupboard Count', '3Light', '40Light',
          '4Light', '50Light', '5Light', '6Light', '7Light', '8Light', '9Light']
fans = ['10Fan', '11Fan', '12Fan', '1Fan', '2Fan', '3Fan', '4Fan', '5Fan', '6Fan', '7Fan', '8Fan', '9Fan']
beds = ['Bed', '1Bed', '2Bed', '3Bed']
geysers = ['Geyser', '1Geyser', '2Geyser', '3Geyser', '4Geyser', '5Geyser', '6Geyser', '7Geyser']
ACs = ['AC', '1AC', '2AC', '3AC', '4AC', '5AC', '6AC', '7AC', '8AC']
wardrobes = ['1Wardrobe', '2Wardrobe', '3Wardrobe', '4Wardrobe', '5Wardrobe', '6Wardrobe', '8Wardrobe']
TV = ['1TV', 'TV']

for pos, name, cols in zip(range(0,7), ["Light(s)", "Fan(s)", "Bed(s)", "Geyser(s)", "Wardrobe(s)", "AC(s)",
                                        "TV(s)"], 
                           [lights, fans, beds, geysers, wardrobes, ACs, TV]):
    
    Facility_data.insert(pos, name, Facility_data[cols].sum(axis = 1), allow_duplicates=False)
    Facility_data.drop(columns = cols, inplace=True)


electrificaton = ['Electrification(Transforme', 'olar Energy etc)']
fire = ['Fire Fighting System', 'Fire Sprinklers']
gym = ['Gym', 'Gymnasium', 'Fitness Center']
lift = ['Lift', 'Lift(s)']
medical = ['Medical Store/Pharmacy', 'Health Facilities']
sports = ['Sports', 'Sports Area', 'Sports Facility']
vaastu = ['Vaastu Compliant', 'Has vastu compliant', 'Vastu Compliant']
water_conserve = ['Water Conservatio', 'ain water Harvesting']
garbage = ['Garbage Disposal', 'Solid Waste Management And Disposal']

for pos, name, cols in zip(range(7,16), ['Electrificaton', 'Fire System', 'Gym Facility', 'Lift System(s)', 
                                         'Medical Facilities', 'Sports Facilities', 'Vaastu Compliance', 
                                         'Water Conservation', 'Garbage'], 
                            [electrificaton, fire, gym, lift, medical, sports, vaastu, water_conserve, garbage]):
    
    Facility_data.insert(pos, name, Facility_data[cols].sum(axis = 1), allow_duplicates=False)
    Facility_data.drop(columns = cols, inplace=True)

In [182]:
Facility_data.head(2)

,Light(s),Fan(s),Bed(s),Geyser(s),Wardrobe(s),AC(s),TV(s),Electrificaton,Fire System,Gym Facility,Lift System(s),Medical Facilities,Sports Facilities,Vaastu Compliance,Water Conservation,Garbage,24X7 Water Supply,24x7 Security,Amphitheater,Badminton Court,Banquet Hall,Bar/Chill-out Lounge,Barbecue Area,Billiards / Snooker Table,Bus Shelter,Business Suites,CCTV,Car Parking,Card Room,Central Cooling System,Changing Room,Children's Play Area,Chimney,Closed Car Parking,Club House,Club Rooftop,Community Buildings,Community Hall,Concierge Service,Creche/Day Care,Cricket Pitch,Cupboard,Cycling & Jogging Track,Dining Table,Entrance Lobby,Fire Alarm,Flower Garden,Food Court,Football Field,Fountains,Fridge,Garden,Gas Pipeline,Gated Community,Gazebo,Grocery Shop,Hockey Ground,Indoor Games,Intercom,Internet / Wi-Fi,Jogging Track,Kid's Pool,Kids Area,Landscaping & Tree Planting,Letter Box,Maintenance Staff,Microwave,Mini Theatre,Modular Kitchen,Multipurpose Room,Natural Pond,Parking,Partial Power Backup,Paved Compound,Pet Grooming,Pet allowed,Power Backup,RO Water System,Rain Water Harvesting,Reading Lounge,Recreation Facilities,Reflexology Park,Regular Water Supply,Sauna Bath,School,Security Cabin,Senior Citizen Siteout,Servant Room,Sewage Treatment Plant,Shopping Mall,Skating Rink,Sofa,Solar Lighting,Spa,Squash Court,Stove,Street Lighting,Sun Bathing,Swimming Pool,Table Tennis,Temple,Tennis Court,Theme Park,Valet Parking,Video Door Security,Visitor Parking,Volleyball Court,Waiting Lounge,Washing Machine,Water Purifier,Water Softner Plant,Water Storage,Yoga / Meditation Area
Society_Loc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"AIPL The Peaceful Homes, Sector 70A",4,6,0,2,3,4,0,0,0,5,3,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,1,0,0,0,0,0,0,6,5,3,0,0,0,0,0,0,4,2,8,0,0,0,0,5,2,0,0,0,1,4,0,0,3,0,0,5,0,1,0,0,0,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,3,0,0,0,11,0,0,7,0,0,0,0,0,0,0,0,0,3,6,0,0,0
"AIPL Zen Residences, Sector 70A",5,5,0,0,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [192]:
## Scaling Features:
scaler = StandardScaler()
scaled_facility_data = pd.DataFrame(scaler.fit_transform(Facility_data),  # converting the scaled data into DF...
                                    index = Facility_data.index, columns = Facility_data.columns)    

scaled_facility_data.sample(3)

,Light(s),Fan(s),Bed(s),Geyser(s),Wardrobe(s),AC(s),TV(s),Electrificaton,Fire System,Gym Facility,Lift System(s),Medical Facilities,Sports Facilities,Vaastu Compliance,Water Conservation,Garbage,24X7 Water Supply,24x7 Security,Amphitheater,Badminton Court,Banquet Hall,Bar/Chill-out Lounge,Barbecue Area,Billiards / Snooker Table,Bus Shelter,Business Suites,CCTV,Car Parking,Card Room,Central Cooling System,Changing Room,Children's Play Area,Chimney,Closed Car Parking,Club House,Club Rooftop,Community Buildings,Community Hall,Concierge Service,Creche/Day Care,Cricket Pitch,Cupboard,Cycling & Jogging Track,Dining Table,Entrance Lobby,Fire Alarm,Flower Garden,Food Court,Football Field,Fountains,Fridge,Garden,Gas Pipeline,Gated Community,Gazebo,Grocery Shop,Hockey Ground,Indoor Games,Intercom,Internet / Wi-Fi,Jogging Track,Kid's Pool,Kids Area,Landscaping & Tree Planting,Letter Box,Maintenance Staff,Microwave,Mini Theatre,Modular Kitchen,Multipurpose Room,Natural Pond,Parking,Partial Power Backup,Paved Compound,Pet Grooming,Pet allowed,Power Backup,RO Water System,Rain Water Harvesting,Reading Lounge,Recreation Facilities,Reflexology Park,Regular Water Supply,Sauna Bath,School,Security Cabin,Senior Citizen Siteout,Servant Room,Sewage Treatment Plant,Shopping Mall,Skating Rink,Sofa,Solar Lighting,Spa,Squash Court,Stove,Street Lighting,Sun Bathing,Swimming Pool,Table Tennis,Temple,Tennis Court,Theme Park,Valet Parking,Video Door Security,Visitor Parking,Volleyball Court,Waiting Lounge,Washing Machine,Water Purifier,Water Softner Plant,Water Storage,Yoga / Meditation Area
Society_Loc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"M3M Golf Hills, Sector 79",-0.794568,-0.80177,-0.496929,-0.394640,-0.736962,-0.269014,-0.453518,-0.094864,-0.415687,-0.846679,-0.837931,-0.115849,-0.714520,-0.574381,-0.139937,-0.160043,-0.259708,-0.089443,-0.500346,-0.138934,-0.080322,-0.057831,-0.057831,-0.057831,-0.057831,-0.077178,-0.410426,-0.332981,-0.057831,-0.104022,-0.24991,-0.278489,-0.506827,-0.092981,-0.257209,-0.057831,-0.093118,-0.27428,-0.104923,-0.060802,-0.389768,-0.975436,-0.07526,-0.713488,-0.093743,-0.104923,-0.057831,-0.057831,-0.057831,-0.360766,-0.530083,-0.802139,-0.366761,-0.361439,-0.293072,-0.293412,-0.057831,-0.105411,-0.756858,-0.259317,-0.145994,-0.19276,-0.627470,-0.500130,-0.057831,-0.100504,-0.584892,-0.246635,-0.419415,-0.195792,-0.105684,-0.335934,-0.057831,-0.057831,-0.057831,-0.840134,-0.805529,-0.057831,-0.081923,-0.103526,-0.102598,-0.142907,-0.286224,-0.226003,-0.145865,-0.077693,-0.118254,-0.654523,-0.07526,-0.06754,-0.391004,-0.721522,-0.160241,-0.374380,-0.115641,-0.627949,-0.073225,-0.057831,-0.820618,-0.335359,-0.057831,-0.406861,-0.109764,-0.143177,-0.070186,-0.078014,-0.211017,-0.223746,-0.552825,-0.897638,-0.175119,-0.137050,-0.15386
"Godrej Icon, Sector 88A",-0.092446,-0.13915,0.695700,0.262181,0.408575,0.160264,-0.453518,-0.094864,-0.415687,0.035891,-0.051879,-0.115849,0.119550,1.431865,-0.139937,-0.160043,-0.259708,-0.089443,-0.500346,-0.138934,-0.080322,-0.057831,-0.057831,-0.057831,-0.057831,-0.077178,-0.410426,-0.332981,-0.057831,-0.104022,-0.24991,-0.278489,0.289237,-0.092981,-0.257209,-0.057831,-0.093118,-0.27428,-0.104923,-0.060802,1.844570,0.009853,-0.07526,0.203199,-0.093743,-0.104923,-0.057831,-0.057831,-0.057831,-0.360766,0.281269,0.035167,0.208550,-0.361439,2.859493,-0.293412,-0.057831,-0.105411,0.210991,-0.259317,-0.145994,-0.19276,-0.126162,-0.500130,-0.057831,-0.100504,-0.063443,-0.246635,-0.419415,-0.195792,-0.105684,-0.335934,-0.057831,-0.057831,-0.057831,-0.053000,-0.279039,-0.057831,-0.081923,-0.103526,-0.102598,3.677384,-0.286224,7.038364,-0.145865,-0.077693,-0.118254,-0.117295,-0.07526,-0.06754,-0.391004,1.469335,-0.160241,2.410264,-0.115641,-0.453519,-0.073225,-0.057831,1.088853,-0.335359,-0.057831,-0.406861,-0.109764,-0.143177,-0.070186,-0.078014,-0.211017,-0.223746,0.309467,-0.002238,-0.175119,-0.137050,-0.15386
"Apex Our

<br><br>
### **Creating "Society Similarity Matrix" using Cosine Similerity Score between Societies:**

In [195]:
## Computing Similarity Score between Socities based on Scaled Facilities Information:
Facility_similarity_matrix = pd.DataFrame(cosine_similarity(scaled_facility_data, scaled_facility_data),
                                          index = scaled_facility_data.index, 
                                          columns = scaled_facility_data.index)
Facility_similarity_matrix.head(4)

Society_Loc,"AIPL The Peaceful Homes, Sector 70A","AIPL Zen Residences, Sector 70A","ATS Kocoon, Sector 109","ATS Marigold, Sector 89A","ATS Tourmaline, Sector 109","ATS Triumph, Sector 104","AVL 36, Sector 36","Adani Aangan, Sector 89A","Adani Brahma Samsara Vilasa, Sector 63","Adani M2K Oyster Grande, Sector 102","Adani Oyster Greens, Sector 102","Ambience Creacions, Sector 22A","Ambience Lagoon, DLF Phase 3","Anant Raj Maceo, Sector 91","Ansal Heights 86, Sector 86","Ansal Heights, Sector 92","Ansal Sushant Estate, Sector 52","Ansal Valley View Estate, Gwal Pahari","Ansals Highland Park, Sector 103","Apex Our Homes 1, Sector 37C","Apex Our Homes, Sector 37C","Ardee Palm Grove Heights, Sector 52","Ashiana Anmol Plaza Phase I, Sector 33","Ashiana Anmol, Dhunela","Ashiana Landcraft The Center Court, Sector 88A","Ashiana Mulberry, Sector 2","Assotech Blith, Sector 99","BPTP Amstoria Country Floor, Sector 102","BPTP Park Serene, Sector 37D","BPTP Terra, Sector 37D","Bestech Altura, Sector 79","Bestech Park View Altura, Sector 79","Bestech Park View Ananda, Sector 81","Bestech Park View Grand Spa, Sector 81","Bestech Park View Residency, Palam Vihar","Bestech Park View Sanskruti, Sector 92","Bestech Park View Spa Next, Sector 67","Bhagwanti Apartment, Sector 56","Birla Navya Gurugram, Kadarpur","Breez Global Heights, Sector 33","Brisk Lumbini Terrace Homes, Sector 109","CGHS City Heights, Sector 39","CGHS Millenium Residency, Sector 47","CGHS Progressive Apartments, Sector 55","CGHS Sanskriti Apartments, Sector 43","CGHS Tarika CGHS, Sushant Lok Phase 1","CGHS The New Greenwood, Sector 52","CGHS Viman Apartment, Sector 9A","CHD Avenue 71, Sector 72","Central Aqua Front Towers, Sector 33","Central Park Aqua Front Towers, Sector 33","Central Park Flamingo Floors, Sector 32","Central Park Flower Valley, Sector 33","Chintels Serenity Phase 1 Tower 3 4 5 Pocket B, Sector 109","Chintels Serenity Pocket B Phase II, Sector 109","Conscient Elevate, Sector 59","Conscient Habitat, Sector 99A","Conscient Heritage Max, Sector 102","Conscient Hines Elevate, Sector 59","Corona Optus, Sector 37C","Cosmos Express 99, Sector 99","DLF Carlton Estate, Sector 53","DLF Exclusive Floors, Sector 54","DLF Express Greens, Manesar","DLF Garden City Enclave Floors, Sector 93","DLF Garden City Independent Floors, Sector 92","DLF New Town Heights 1, Sector 86","DLF New Town Heights 1, Sector 90","DLF New Town Heights 2, Sector 86","DLF New Town Heights 3, Sector 91","DLF Oakwood Estate, Sector 25","DLF Park Place, DLF Phase 5","DLF Regal Gardens, Sector 90","DLF Regency Park 1, Sector 43","DLF Richmond Park, Sector 43","DLF Ridgewood Estate, DLF Phase 4","DLF Silver Oaks, DLF Phase 1","DLF The Crest, DLF Phase 5","DLF The Primus, Sector 82A","DLF The Skycourt, Sector 86","DLF The Ultima Phase II, Sector 81","DLF The Ultima, Sector 81","DLF Wellington Estate, Sector 53","DLF Westend Heights, Sector 53","Eldeco Acclaim, Sector 2","Eldeco Accolade, Sector 2","Emaar Digi Homes, Sector 62","Emaar Emerald Classic, Sector 65","Emaar Emerald Estate, Sector 65","Emaar Emerald Floors Premier, Sector 65","Emaar Emerald Floors, Sector 65","Emaar Emerald Hill, Sector 65","Emaar Gurgaon Greens, Sector 102","Emaar Imperial Gardens, Sector 102","Emaar MGF The Palm Drive, Sector 66","Emaar Palm Gardens, Sector 83","Emaar Palm Heights, Sector 77","Emaar Palm Hills, Sector 77","Emaar Palm Premier, Sector 77","Emaar Palm Select, Sector 77","Emaar Palm Terraces Select, Sector 66","Emaar The Enclave, Sector 66","Experion The Heartsong, Sector 108","Experion Windchants, Sector 112","GLS Arawali Homes, Sector 4","GLS Avenue 51, Sector 92","GLS South Avenue, Sector 92","GPL Eden Heights, Sector 70","Godrej 101, Sector 79","Godrej Air Phase I, Sector 85","Godrej Air, Sector 85","Godrej Aria, Sector 79","Godrej Frontier, Sector 80","Godrej Icon, Sector 88A","Godrej Meridien, Sector 106","Godrej Nature Plus, Sector 33","Godrej Oasis, Sector 88A","Godrej Summit, Sector 104","Gurgaon One 

<br><br>
### **Finally Recommending Societies based on Facility Similarity Scores:**

In [198]:
## Using the above Pricing Similarity Matrix for Recommending Top 10 Similar Socities:
def recommend_societies_by_facility(Society_name, Facility_similarity_matrix):
    societies = Facility_similarity_matrix[Society_name].sort_values(ascending = False).iloc[1:11].index
    return pd.DataFrame(societies, index = range(1,11))

recommend_societies_by_facility("ATS Kocoon, Sector 109", Facility_similarity_matrix)

,Society_Loc
1,"Tulip White, Sector 69"
2,"Orris Greenopolis, Sector 89"
3,"Satya The Hermitage, Sector 103"
4,"Bestech Park View Sanskruti, Sector 92"
5,"Tata Primanti, Southern Peripheral Road"
6,"ATS Triumph, Sector 104"
7,"DLF The Ultima Phase II, Sector 81"
8,"M3M Skywalk, Sector 74"
9,"Emaar Emerald Hill, Sector 65"
10,"M3M Woodshire, Sector 107"


In [200]:
## Saving the Similarity Matrix as CSV:
Facility_similarity_matrix.to_csv("REC_2_Facility_similarity_matrix.csv")

<br><br><br>

---

# `III.` **Location Based Society Recommendation**

In [263]:
def locations_info_scrapper(data):
    Soc_locations = {}                              # Dictionary of locations with count around the society...

    def loc_counts(data):
        for location in data:
            if location not in Soc_locations:
                Soc_locations[location] = 1
            else:
                Soc_locations[location] += 1

    values = data['Nearby_landmarks'].str.replace("/", "").str.replace('"', "'") # Fixing irregular parts of string..
    
    locations_lists = values.apply(lambda x: [i.split("', '")[1] for i in x[3:-3].split("'], ['")])
    locations_lists.apply(loc_counts)

    
    Soc_Record = pd.Series(Soc_locations)     # Series to store overall locations with count around the society...
    
    return pd.DataFrame(Soc_Record)


Locations_data = df.groupby(by = 'Society_Loc').apply(locations_info_scrapper).unstack(fill_value=0)
Locations_data.columns = [i[1] for i in Locations_data.columns.values]

Locations_data.sample(2)

,Aarvy Healthcare Pvt Ltd.,Bharti International Convent School | Best School in Gurgaon,Bikaner sweets,Bikanervala,CD International School,Canwinn Aarogya Dham (A unit of CanWinn Foundation),Cyber City Metro Station,DAV Public School,Delhi Public School,Delhi Public School Sushant Lok | Best School in Gurgaon,"Delhi Public School, Sector 84","Euro International School, Sector 37D, Gurugram","Euro International School, Sector 84, Gurugram","Euro International School, Sector- 109.",GD GOENKA SIGNATURE SCHOOL,GD Goenka Public School,Haldiram's - Kherki Daula,Harish Bakers & Sweets Sector 83,Huda City Centre Metro Station,Iffco Chowk Metro Station,Jai Sai Ram Hospital,Kunskapsskolan,Lady Florence Public School,"Lotus Valley International School, Gurgaon",MG Road Metro Station,Manav Rachna International School,"Manipal Hospital, Palam Vihar, Gurugram","Marengo Asia Hospitals, Gurugram",McDonald's India Gurugram,Medanta Gurgaon,Millennium City Centre Gurugram Metro Station,Miracles Apollo CradleSpectra Sec 82 - Hospital,Moulsari Avenue Metro Station,"Narayana Hospital, Gurugram","OM Sweets, Sector 10, Gurugram","OM Sweets, Sector 4, Gurugram",Ompee Global School,PIZZA KING,"Paras Hospitals, Gurgaon",Park Hospital,"Park Hospital, Palam Vihar",Phase 1 Metro Station,Phase 2 Metro Station,Polaris Hospital,"Prudence Schools - Top & Best CBSE School in Dwarka, Delhi",R L International School,RPS International School,Rabindranath World School,Rotary Public School,Royal Public Senior Secondary School,"Ryan International School, Sohna Road, Bhondsi, Gurugram - CBSE",SCR Public School,Saraswati Model School,Sector 42-43 Metro Station,Sector 53-54 Metro Station,Sector 54 Chowk Metro Station,Sector 55-56 Metro Station,Shalom Hills International School,Shiksha Bharti Public School.,Shri Shiv Narayan Sidheswar Senior Secondary Public School,St. Xavier's High School,St.PBN Public School,Suraj School,THE SIGNATURE ADVANCED SUPER SPECIALITY HOSPITAL,Tandoori snipers family restaurant and cafe,The Maurya School | Best School In Palam Vihar | Top School in Palam Vihar,The Shikshiyan School,The Vivekananda School,The Vivekananda School - Sector 69 | Senior Branch,Unicosmos School,Vardaan Hospital & Trauma Centre,Vasudeva Restaurant,Vega Schools,Yashobhoomi Dwarka Sector 25 Metro Station,Yashroop Hospital | Best Maternity & Multispeciality Hospital in Gurgaon | Best Orthopedic Hospital in Gurgaon
Society_Loc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Ashiana Anmol, Dhunela",0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0
"Godrej Oasis, Sector 88A",0,0,0,0,0,0,0,0,0,0,0,26,0,0,0,0,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0,0,0,0,0,0,0,0,0,0,0


In [267]:
## Scaling Features:
scaler = MinMaxScaler()
scaled_locations_data = pd.DataFrame(scaler.fit_transform(Locations_data),  # converting the scaled data into DF...
                                    index = Locations_data.index, columns = Locations_data.columns)    

scaled_locations_data.sample(3)

,Aarvy Healthcare Pvt Ltd.,Bharti International Convent School | Best School in Gurgaon,Bikaner sweets,Bikanervala,CD International School,Canwinn Aarogya Dham (A unit of CanWinn Foundation),Cyber City Metro Station,DAV Public School,Delhi Public School,Delhi Public School Sushant Lok | Best School in Gurgaon,"Delhi Public School, Sector 84","Euro International School, Sector 37D, Gurugram","Euro International School, Sector 84, Gurugram","Euro International School, Sector- 109.",GD GOENKA SIGNATURE SCHOOL,GD Goenka Public School,Haldiram's - Kherki Daula,Harish Bakers & Sweets Sector 83,Huda City Centre Metro Station,Iffco Chowk Metro Station,Jai Sai Ram Hospital,Kunskapsskolan,Lady Florence Public School,"Lotus Valley International School, Gurgaon",MG Road Metro Station,Manav Rachna International School,"Manipal Hospital, Palam Vihar, Gurugram","Marengo Asia Hospitals, Gurugram",McDonald's India Gurugram,Medanta Gurgaon,Millennium City Centre Gurugram Metro Station,Miracles Apollo CradleSpectra Sec 82 - Hospital,Moulsari Avenue Metro Station,"Narayana Hospital, Gurugram","OM Sweets, Sector 10, Gurugram","OM Sweets, Sector 4, Gurugram",Ompee Global School,PIZZA KING,"Paras Hospitals, Gurgaon",Park Hospital,"Park Hospital, Palam Vihar",Phase 1 Metro Station,Phase 2 Metro Station,Polaris Hospital,"Prudence Schools - Top & Best CBSE School in Dwarka, Delhi",R L International School,RPS International School,Rabindranath World School,Rotary Public School,Royal Public Senior Secondary School,"Ryan International School, Sohna Road, Bhondsi, Gurugram - CBSE",SCR Public School,Saraswati Model School,Sector 42-43 Metro Station,Sector 53-54 Metro Station,Sector 54 Chowk Metro Station,Sector 55-56 Metro Station,Shalom Hills International School,Shiksha Bharti Public School.,Shri Shiv Narayan Sidheswar Senior Secondary Public School,St. Xavier's High School,St.PBN Public School,Suraj School,THE SIGNATURE ADVANCED SUPER SPECIALITY HOSPITAL,Tandoori snipers family restaurant and cafe,The Maurya School | Best School In Palam Vihar | Top School in Palam Vihar,The Shikshiyan School,The Vivekananda School,The Vivekananda School - Sector 69 | Senior Branch,Unicosmos School,Vardaan Hospital & Trauma Centre,Vasudeva Restaurant,Vega Schools,Yashobhoomi Dwarka Sector 25 Metro Station,Yashroop Hospital | Best Maternity & Multispeciality Hospital in Gurgaon | Best Orthopedic Hospital in Gurgaon
Society_Loc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Vipul Lavanya Apartments Tower 2 And 3, Sector 81",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.000000,0.128571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.176471,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Signature Global The Millennia I, Sector 37D",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.224138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.191176,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.113043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Signature Global The Roselia, Sector 95A",0.0,0.0,0.0,0.771429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.385714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.385714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<br><br>
### **Creating "Society Similarity Matrix" using Cosine Similerity Score between Societies:**

In [270]:
## Computing Similarity Score between Socities based on Scaled Facilities Information:
Locations_similarity_matrix = pd.DataFrame(cosine_similarity(scaled_locations_data, scaled_locations_data),
                                          index = scaled_locations_data.index, 
                                          columns = scaled_locations_data.index)
Locations_similarity_matrix.head(4)

Society_Loc,"AIPL The Peaceful Homes, Sector 70A","AIPL Zen Residences, Sector 70A","ATS Kocoon, Sector 109","ATS Marigold, Sector 89A","ATS Tourmaline, Sector 109","ATS Triumph, Sector 104","AVL 36, Sector 36","Adani Aangan, Sector 89A","Adani Brahma Samsara Vilasa, Sector 63","Adani M2K Oyster Grande, Sector 102","Adani Oyster Greens, Sector 102","Ambience Creacions, Sector 22A","Ambience Lagoon, DLF Phase 3","Anant Raj Maceo, Sector 91","Ansal Heights 86, Sector 86","Ansal Heights, Sector 92","Ansal Sushant Estate, Sector 52","Ansal Valley View Estate, Gwal Pahari","Ansals Highland Park, Sector 103","Apex Our Homes 1, Sector 37C","Apex Our Homes, Sector 37C","Ardee Palm Grove Heights, Sector 52","Ashiana Anmol Plaza Phase I, Sector 33","Ashiana Anmol, Dhunela","Ashiana Landcraft The Center Court, Sector 88A","Ashiana Mulberry, Sector 2","Assotech Blith, Sector 99","BPTP Amstoria Country Floor, Sector 102","BPTP Park Serene, Sector 37D","BPTP Terra, Sector 37D","Bestech Altura, Sector 79","Bestech Park View Altura, Sector 79","Bestech Park View Ananda, Sector 81","Bestech Park View Grand Spa, Sector 81","Bestech Park View Residency, Palam Vihar","Bestech Park View Sanskruti, Sector 92","Bestech Park View Spa Next, Sector 67","Bhagwanti Apartment, Sector 56","Birla Navya Gurugram, Kadarpur","Breez Global Heights, Sector 33","Brisk Lumbini Terrace Homes, Sector 109","CGHS City Heights, Sector 39","CGHS Millenium Residency, Sector 47","CGHS Progressive Apartments, Sector 55","CGHS Sanskriti Apartments, Sector 43","CGHS Tarika CGHS, Sushant Lok Phase 1","CGHS The New Greenwood, Sector 52","CGHS Viman Apartment, Sector 9A","CHD Avenue 71, Sector 72","Central Aqua Front Towers, Sector 33","Central Park Aqua Front Towers, Sector 33","Central Park Flamingo Floors, Sector 32","Central Park Flower Valley, Sector 33","Chintels Serenity Phase 1 Tower 3 4 5 Pocket B, Sector 109","Chintels Serenity Pocket B Phase II, Sector 109","Conscient Elevate, Sector 59","Conscient Habitat, Sector 99A","Conscient Heritage Max, Sector 102","Conscient Hines Elevate, Sector 59","Corona Optus, Sector 37C","Cosmos Express 99, Sector 99","DLF Carlton Estate, Sector 53","DLF Exclusive Floors, Sector 54","DLF Express Greens, Manesar","DLF Garden City Enclave Floors, Sector 93","DLF Garden City Independent Floors, Sector 92","DLF New Town Heights 1, Sector 86","DLF New Town Heights 1, Sector 90","DLF New Town Heights 2, Sector 86","DLF New Town Heights 3, Sector 91","DLF Oakwood Estate, Sector 25","DLF Park Place, DLF Phase 5","DLF Regal Gardens, Sector 90","DLF Regency Park 1, Sector 43","DLF Richmond Park, Sector 43","DLF Ridgewood Estate, DLF Phase 4","DLF Silver Oaks, DLF Phase 1","DLF The Crest, DLF Phase 5","DLF The Primus, Sector 82A","DLF The Skycourt, Sector 86","DLF The Ultima Phase II, Sector 81","DLF The Ultima, Sector 81","DLF Wellington Estate, Sector 53","DLF Westend Heights, Sector 53","Eldeco Acclaim, Sector 2","Eldeco Accolade, Sector 2","Emaar Digi Homes, Sector 62","Emaar Emerald Classic, Sector 65","Emaar Emerald Estate, Sector 65","Emaar Emerald Floors Premier, Sector 65","Emaar Emerald Floors, Sector 65","Emaar Emerald Hill, Sector 65","Emaar Gurgaon Greens, Sector 102","Emaar Imperial Gardens, Sector 102","Emaar MGF The Palm Drive, Sector 66","Emaar Palm Gardens, Sector 83","Emaar Palm Heights, Sector 77","Emaar Palm Hills, Sector 77","Emaar Palm Premier, Sector 77","Emaar Palm Select, Sector 77","Emaar Palm Terraces Select, Sector 66","Emaar The Enclave, Sector 66","Experion The Heartsong, Sector 108","Experion Windchants, Sector 112","GLS Arawali Homes, Sector 4","GLS Avenue 51, Sector 92","GLS South Avenue, Sector 92","GPL Eden Heights, Sector 70","Godrej 101, Sector 79","Godrej Air Phase I, Sector 85","Godrej Air, Sector 85","Godrej Aria, Sector 79","Godrej Frontier, Sector 80","Godrej Icon, Sector 88A","Godrej Meridien, Sector 106","Godrej Nature Plus, Sector 33","Godrej Oasis, Sector 88A","Godrej Summit, Sector 104","Gurgaon One 

<br><br>
### **Finally Recommending Societies based on Location Similarity Scores:**

In [273]:
## Using the above Pricing Similarity Matrix for Recommending Top 10 Similar Socities:
def recommend_societies_by_locations(Society_name, Locations_similarity_matrix):
    societies = Locations_similarity_matrix[Society_name].sort_values(ascending = False).iloc[1:11].index
    return pd.DataFrame(societies, index = range(1,11))

recommend_societies_by_locations("ATS Kocoon, Sector 109", Locations_similarity_matrix)

,Society_Loc
1,"Sobha City, Sector 108"
2,"Godrej Meridien, Sector 106"
3,"Chintels Serenity Pocket B Phase II, Sector 109"
4,"Brisk Lumbini Terrace Homes, Sector 109"
5,"Zara Rossa, Sector 112"
6,"Paras Dews, Sector 106"
7,"Viridian The Longevity Project At Plaza 106, S..."
8,"ATS Tourmaline, Sector 109"
9,"Experion Windchants, Sector 112"
10,"Raheja Atharva, Sector 109"


In [278]:
## Saving the Similarity Matrix as CSV:
Locations_similarity_matrix.to_csv("REC_3_Locations_similarity_matrix.csv")

<br><br>

---
---

# ⭐ RECOMMENDATION ENGINE

### **`Loading the 3 Similarity Matrices & Creating a Weighted Society Recommendation Engine`**
- The **Recommendation Engine** will **suggest Society based on 3 considerartions**:
    1. **Property Pricing**
    2. **Society Amenities or Facilities**
    3. **Surrounding Locations**

In [102]:
pricing = pd.read_csv("REC_1_Pricing_similarity_matrix.csv", index_col = 'Society_Loc')
facilities = pd.read_csv("REC_2_Facility_similarity_matrix.csv", index_col = 'Society_Loc')
locations = pd.read_csv("REC_3_Locations_similarity_matrix.csv", index_col = 'Society_Loc')

## Function to recommend Top 10 similar Societies:
def recommend_society(society_name, pricing, facilities, locations):
    sim_matrix = (15 * pricing) + (5 * facilities) + (5 * locations) # We can set the weights as per the preference
    
    societies = sim_matrix[society_name].sort_values(ascending = False).iloc[1:11].index
    return pd.DataFrame(societies, index = range(1,11))

    
m = recommend_society("Godrej Meridien, Sector 106", pricing, facilities, locations)
m

,Society_Loc
1,"Paras Dews, Sector 106"
2,"Sobha City, Sector 108"
3,"Godrej Air, Sector 85"
4,"M3M Skywalk, Sector 74"
5,"M3M Skycity, Sector 65"
6,"Birla Navya Gurugram, Kadarpur"
7,"Spaze Privy, Sector 72"
8,"Hero Homes, Sector 104"
9,"Brisk Lumbini Terrace Homes, Sector 109"
10,"M3M Merlin, Sector 67"


<br><br><br>

----

### **Saving Data as CSV file to use for Streamlit App:**

In [161]:
data_df = df[['Society_Loc', 'Built_up_area_in_sqft', 'Avg_price_rupee_per_sqft', 'Price_in_rupees', 'Flat',
              'Society', 'Sector_Locality', 'Locality', 'Built_Up_Area', 'Avg_Price', 'Price','Floor', 
              'Facing', 'Brokerage', 'EMI', 'Link']]

data_df.to_csv("Gurgaon_housing_data_of_top_300_soc_for_recommendation.csv")

<br><br><br>

---
---